In [ ]:
!wget -O git-lfs.tar.gz https://github.com/git-lfs/git-lfs/releases/download/v2.4.2/git-lfs-linux-amd64-2.4.2.tar.gz
!tar -xvzf git-lfs.tar.gz
!./git-lfs-2.4.2/install.sh
!git lfs clone https://github.com/mozilla/deepspeech
%cd deepspeech
!git checkout -f tags/v0.7.3
%cd ..

In [ ]:

#download the requirements

!pip3 install -r deepspeech/requirements_transcribe.txt
!pip3 install -r deepspeech/requirements_tests.txt
!pip3 install -r deepspeech/requirements_eval_tflite.txt
!pip3 install deepspeech-gpu==0.7.3

!pip3 uninstall tensorflow -y
!pip3 install tensorflow-gpu==1.15.2
!pip3 install ds_ctcdecoder==0.7.3
%cd deepspeech

!pip3 install -e .  # install reqs if missed any
%cd ..

In [ ]:
# download sample data for testing the deepspeech model

!wget https://www.dropbox.com/sh/7wp15mwmuv6exdj/AAAb7LJE0jBGyJ0titPZ1jDZa?dl=0
!mv 'AAAb7LJE0jBGyJ0titPZ1jDZa?dl=0' data.tar
!unzip data.tar
!ls

In [ ]:
#download pretrained model and scorer
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.pbmm
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.scorer

In [ ]:
import tensorflow as tf
from google.colab import drive

# 使用工具colab的接口挂载google drive目录，这样可以从外部获取数据并且可以把训练好的模型保存在google drive上
drive.mount('/content/gdrive')
tf.test.gpu_device_name()

Mounted at /content/gdrive


'/device:GPU:0'

In [ ]:
!pip3 install jiwer
from jiwer import wer

     |████████████████████████████████| 50 kB 4.1 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149865 sha256=e56c46ea2d7f76dd5275abd465f487cd2d46b2bf9a94301ba805304e57e78ef1
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:

from deepspeech import Model
import scipy.io.wavfile as wav
import pandas as pd
dsModel = Model("deepspeech-0.7.0-models.pbmm")


In [ ]:
path="sample.wav"

fs, audio = wav.read(path)
output = dsModel.stt(audio)
print("Deepspeech Transcription: ", output)
#print("Actual Transcription: ", row['Transcription'])
print("-----------------------------------------------------------------------------------")

Deepspeech Transcription:  this little work was finished in the year eighteen o three and intended for a mediate publication it was disposed of to a bookseller it was even advertised
-----------------------------------------------------------------------------------


In [ ]:
#import speech_recognition as sr
from os import path
import os.path
import operator
from os import system
from os import listdir
from os.path import isfile, join
import wave
import scipy as sc
import librosa
import IPython.display as ipd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
%matplotlib inline
import math
import librosa as lb
import scipy
from sklearn.decomposition import PCA
import pandas as pd
from os import listdir
from os.path import isfile, join
import time
from itertools import product
import datetime
import sys
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from deepspeech import Model
from scipy.io import wavfile

def transcribe_deepspeech(my_path):
    #audio_path = 'audio/2830-3980-0043.wav'
    audio_path=my_path
    fs, data = wavfile.read(audio_path)
    data=data.astype(np.int16)
    #model_path = "C:/Users/wuxia/ad_samples/deepspeech-0.6.0-models/output_graph.pb" # 已下载的模型地址（正确的模型文件中有以.pb结尾的文件）
    #ars = Model(model_path,2048) # 1024应该是指窗长，这个是在源码中看到了
    dsModel = Model("deepspeech-0.7.0-models.pbmm")
    translate_txt = dsModel.stt(data)

    print(translate_txt)
    return translate_txt

In [ ]:
def create_wav(name,data):
    
    #url_wav='/content/gdrive/MyDrive/deepspeech/audio_temp/'+name+'temp.wav'
    url_wav=name
    wavfile.write(url_wav, 16000, data)
    return url_wav

In [ ]:
import os
def framing(name,n,data):
    
    transcription=[]

    for i in range(0,n):
        temp=np.copy(data)
        if i< n-1:
            for j in range(512):
                temp[i*512+j]=0
        if i==n-1:
            for j in range(2048):
                temp[i*512+j]=0
        #str_="audio/sample4507_framing/frame_%d.wav"
        str_=create_wav(name,temp)
        #scipy.io.wavfile.write(str_, fs, temp)
        trans_x=transcribe_deepspeech(str_)
        print("%d th frame is set to all 0s." %i)
        print(trans_x)
        transcription.append(trans_x)
        print(transcription)
    return transcription

In [ ]:
import os
#import wave
from jiwer import wer
import numpy as np

path="sample.wav"

fs, data = scipy.io.wavfile.read(path)
print(data.shape)
elementsInBucket = 2048
n = int((len(data)-2048+512)/512)
print(n)
name,category=os.path.splitext("sample.wav")
    
#name,category=os.path.splitext(filepath+filename)#分解文件扩展名

print(name)

url_wav_2=name+'_temp.wav'
transcription=framing(url_wav_2,n,data)




indexlist=[]

for i in range(len(transcription)):
  if(wer(output, transcription[i])>0):
    indexlist.append(i)



np.save("index.npy",np.array(indexlist))


In [ ]:
import os
df=pd.read_csv('sample_data.csv')  # read the csv file
for i,row in df.iterrows():
  # resample the input file to 16KHz
  command="ffmpeg -i "+row['File_Path'] +"-ar 16000 -acodec pcm_s16le -ac 1 "+row['File_Path']
  os.system(command)
  fs, audio = wav.read(row['File_Path'])  # do inference with deepspeech
  output = dsModel.stt(audio)
  print("Deepspeech Transcription: ", output)
  print("Actual Transcription: ", row['Transcription'])
  print("-----------------------------------------------------------------------------------")